### 2021/3/9
### 使用到算法计算视热源
### 所用资料：合成分析数据

import os
import numpy as np
import Ngl, Nio
import json
from geopy.distance import distance
import numpy.ma as ma
import sys
import math
from netCDF4 import Dataset

def all_heating(T,uwind,vwind,pt,omega,dimension):
    #dimension里储存的是维度-（气压/纬度/经度）


    disy = np.array([])
    disx = np.array([])

    p    = dimension[1]
    lat  = dimension[2]
    lon  = dimension[3]
    time = np.arange(61)+1
    T_t = np.gradient(T, time, axis=0)
    #处理坐标信息
    for i in range(0,len(lat)-1):
        disy    =   np.append(disy,distance((lat[i],0),(lat[i+1],0)).m)

    for i in range(1,len(lat)-1):
        disx    =   np.append(disx,distance((lat[i],lon[0]),(lat[i],lon[1])).m)

    location = np.array([0])
    for dddd in range(0, len(lat)-1):
        location = np.append(location, np.sum(disy[:dddd + 1]))

    T_x = ma.array(ma.zeros(T.shape), mask=T.mask)
    T_y = ma.array(ma.zeros(T.shape), mask=T.mask)

    T_y = np.gradient(T,location,axis = 2)
    for latt in range(1,len(lat)-1):
        T_x[:,:,latt,:] = np.gradient(T[:,:,latt,:],disx[latt],axis = 2)

    term2 = uwind*T_x+vwind*T_y

    theta_p = np.gradient(pt,p,axis = 1)
    term3 = T
    for pp in range(0,len(p)):
        term3[:,pp,:,:] = math.pow((pp/1000),0.286)*omega[:,pp,:,:]*theta_p[:,pp,:,:]

    term = 1.004*(T_t+term2+term3)         #kJ/(kg K)
    return term,T_t,T_x,T_y


In [4]:
import os
import numpy as np
import numpy.ma as ma
import Ngl, Nio
from netCDF4 import Dataset
import json
import sys
import math
sys.path.append("/mycode/module/")
from module_writenc import *
from module_sun import *
#from attribute import *
with open("/data/onsetdate.json",'r') as load_f:
    a = json.load(load_f)
    

In [5]:
f = Nio.open_file("/data/composite3.nc")

In [ ]:
w = f.variables["OMEGA"][:]

In [ ]:
u = f.variables["uwind"][:]u = f.variables["uwind"][:]

In [ ]:
v = f.variables["vwind"][:]

In [ ]:
t = f.variables["T"][:]

In [ ]:
lat = f.variables["lat"][:]
lon = f.variables["lon"][:]
lev = f.variables["level"[:]]